In [10]:
import pandas as pd
import numpy as np
from numba import njit, prange

from tqdm.notebook import tqdm

In [11]:
train_csv = pd.read_csv("train.csv").drop(["Unnamed: 0"], axis=1)
val_csv = pd.read_csv("val.csv").drop(["Unnamed: 0"], axis=1)

In [12]:
n_users = 3974;
n_items = 3564;

uir_train = train_csv.values
uir_val = val_csv.values

## Without jit

In [13]:
def predict_no_jit(u, i, params):
    μ, bu, bi, P, Q = params
    k = P.shape[1]

    pred = μ + bu[u] + bi[i] + np.dot(Q[i], P[u])
    
    return pred

def predict_batch_no_jit(uir_mat, params):
    predictions = np.zeros(len(uir_mat))
    for it in prange(uir_mat.shape[0]):
        u, i, _ = uir_mat[it]
        predictions[it] = predict_no_jit(u, i, params)
        
    return np.clip(predictions, 1., 5.)

def fit_funk_svd_no_jit(train_data, val_data, n_users, n_items, k, α1, α2, α3, α4, λ1, λ2, n_iters):
    val_size = len(val_data)
    val_exp = val_data[:, -1]
    
    bu = np.zeros(n_users, np.double)
    bi = np.zeros(n_items, np.double)
    
    P = np.random.normal(0, .1, (n_users, k))
    Q = np.random.normal(0, .1, (n_items, k))
    
    μ = np.mean(train_data[:, 2])
    
    min_val_loss = np.inf
    for it in tqdm(range(n_iters)):
        loss = 0
        for u, i, r in train_data:
            pred = μ + bu[u] + bi[i] + np.dot(P[u], Q[i])
            
            error = r - pred
            
            # Updating
            bu[u] += α1 * (error - λ1*bu[u])
            bi[i] += α2 * (error - λ1*bi[i])
            P[u], Q[i] = P[u] + α3*(error*Q[i] - λ2*P[u]), Q[i] + α4*(error*P[u] - λ2*Q[i])
            
            loss += error**2
        loss = np.sqrt(loss/len(train_data))
            
        val_preds = predict_batch_no_jit(val_data, (μ, bu, bi, P, Q))
        min_val_loss = min(
            min_val_loss,
            np.sqrt(1/val_size * np.sum((val_preds - val_exp)**2)),
        )
    
    print("loss:", loss, "val_loss:", min_val_loss)
    
    return μ, bu, bi, P, Q

In [ ]:
fitted_final_params_no_jit = fit_funk_svd_no_jit(
    uir_train,
    uir_val,
    n_users,
    n_items, 
    k=150, 
    α1=0.005, 
    α2=0.005, 
    α3=0.01, 
    α4=0.01,
    λ1=0.05,
    λ2=0.1, 
    n_iters=75,
)

# 07:52

  0%|          | 0/75 [00:00<?, ?it/s]

## With jit

In [8]:
@njit
def predict(u, i, params):
    μ, bu, bi, P, Q = params
    k = P.shape[1]

    pred = μ + bu[u] + bi[i] + np.dot(Q[i], P[u])
    
    return pred

@njit
def predict_batch(uir_mat, params):
    predictions = np.zeros(len(uir_mat))
    for it in prange(uir_mat.shape[0]):
        u, i, _ = uir_mat[it]
        predictions[it] = predict(u, i, params)
        
    return np.clip(predictions, 1., 5.)

@njit
def fit_funk_svd(
    train_data, 
    val_data, 
    n_users, 
    n_items, 
    k, 
    α1, 
    α2, 
    α3, 
    α4, 
    λ1, 
    λ2, 
    n_iters,
    progress_proxy,
):
    val_size = len(val_data)
    val_exp = val_data[:, -1]
    
    bu = np.zeros(n_users, np.double)
    bi = np.zeros(n_items, np.double)
    
    P = np.random.normal(0, .1, (n_users, k))
    Q = np.random.normal(0, .1, (n_items, k))
    
    μ = np.mean(train_data[:, 2])
    
    min_val_loss = np.inf
    for it in range(n_iters):
        loss = 0
        for u, i, r in train_data:
            pred = μ + bu[u] + bi[i] + np.dot(P[u], Q[i])
            
            error = r - pred
            
            # Updating
            bu[u] += α1 * (error - λ1*bu[u])
            bi[i] += α2 * (error - λ1*bi[i])
            P[u], Q[i] = P[u] + α3*(error*Q[i] - λ2*P[u]), Q[i] + α4*(error*P[u] - λ2*Q[i])
            
            loss += error**2
        loss = np.sqrt(loss/len(train_data))
            
        val_preds = predict_batch(val_data, (μ, bu, bi, P, Q))
        min_val_loss = min(
            min_val_loss,
            np.sqrt(1/val_size * np.sum((val_preds - val_exp)**2)),
        )
        
        progress_proxy.update(1)
    
    print("loss:", loss, "val_loss:", min_val_loss)
    
    return μ, bu, bi, P, Q

In [6]:
from numba_progress import ProgressBar

In [9]:
with ProgressBar(total=75) as progress:
    fitted_final_params = fit_funk_svd(
        uir_train,
        uir_val,
        n_users,
        n_items, 
        k=150, 
        α1=0.005, 
        α2=0.005, 
        α3=0.01, 
        α4=0.01,
        λ1=0.05,
        λ2=0.1, 
        n_iters=75,
        progress_proxy=progress,
    )
    
# 00:13

  0%|                                                    | 0/75 [00:00<?, ?it/s]

loss: 0.7480935195946997 val_loss: 0.8634181761551536
